In [2]:
# Импортируем библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import cluster
from sklearn import feature_selection
import re
import optuna
import xgboost as xgb
from catboost import CatBoostRegressor

In [11]:
# загружаем занные
data = pd.read_csv('data/data_real_property.csv')
data.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [256]:
data['schools_rating'] = data['schools'].apply(lambda x: eval(x)[0].get('rating'))

In [257]:
data['schools_rating'] = data['schools_rating'].astype(str)

In [258]:
data['schools_rating'] = data['schools_rating'].str.replace('[a-zA-Z]','', regex=True)

In [260]:
data['schools_rating'] = data['schools_rating'].str.replace('/10','')

In [262]:
data['schools_rating'] = data['schools_rating'].apply(lambda x: [int(i) for i in re.findall(r'\d+', x)])

In [223]:
#data['schools_rating'] = data['schools_rating'].str.findall(r'\b([0-9])\'')

In [264]:
data['schools_rating']

0         [4, 4, 7, 4, 7]
1                  [4, 4]
2               [8, 4, 8]
3           [9, 9, 10, 9]
4               [4, 5, 5]
               ...       
377180            [10, 5]
377181          [1, 5, 7]
377182             [5, 4]
377183                 []
377184          [5, 4, 3]
Name: schools_rating, Length: 377185, dtype: object

In [240]:
from statistics import mean

In [241]:
data['schools_average_rating'] = data['schools_rating'].apply(lambda x: mean(x) if len(x) > 0 else np.nan)

In [237]:
data['schools_average_rating'] = data['schools_rating'].apply(lambda x: sum(x)/len(x) if len(x) > 0 else np.nan)

In [242]:
data['schools_average_rating']

0         5.200000
1         4.000000
2         6.666667
3         9.000000
4         4.666667
            ...   
377180    5.000000
377181    4.333333
377182    4.500000
377183         NaN
377184    4.000000
Name: schools_average_rating, Length: 377185, dtype: float64

In [184]:
data['schools_distance'] = data['schools'].apply(lambda x: eval(x)[0].get('data').get('Distance'))

In [185]:
data['schools_distance']

0         [2.7 mi, 3.6 mi, 5.1 mi, 4.0 mi, 10.5 mi, 12.6...
1                                  [1.65mi, 1.32mi, 1.01mi]
2                                  [1.19mi, 2.06mi, 2.63mi]
3                           [1.05mi, 0.1mi, 1.05mi, 0.81mi]
4                                  [5.96mi, 3.25mi, 3.03mi]
                                ...                        
377180                                    [32.1 mi, 1.1 mi]
377181                             [10.61mi, 1.42mi, 0.4mi]
377182                                     [0.48mi, 0.73mi]
377183                             [1.3 mi, 0.5 mi, 1.9 mi]
377184                             [0.3 mi, 1.1 mi, 4.1 mi]
Name: schools_distance, Length: 377185, dtype: object

In [186]:
data['schools_distance'] = data['schools_distance'].astype(str)

In [187]:
data['schools_distance'] = data['schools_distance'].str.replace('[a-zA-Z]','', regex=True)

In [189]:
data['schools_distance'] = data['schools_distance'].str.findall(r'\b([0-9]+.[0-9]+)')

In [191]:
data['schools_min_distance'] = data['schools_distance'].apply(lambda x: min(x) if len(x) > 0 else np.nan)

In [192]:
data['schools_min_distance']

0         10.5
1         1.01
2         1.19
3          0.1
4         3.03
          ... 
377180     1.1
377181     0.4
377182    0.48
377183     0.5
377184     0.3
Name: schools_min_distance, Length: 377185, dtype: object

In [190]:
data['schools_distance']

0         [2.7, 3.6, 5.1, 4.0, 10.5, 12.6, 2.7, 3.1]
1                                 [1.65, 1.32, 1.01]
2                                 [1.19, 2.06, 2.63]
3                            [1.05, 0.1, 1.05, 0.81]
4                                 [5.96, 3.25, 3.03]
                             ...                    
377180                                   [32.1, 1.1]
377181                            [10.61, 1.42, 0.4]
377182                                  [0.48, 0.73]
377183                               [1.3, 0.5, 1.9]
377184                               [0.3, 1.1, 4.1]
Name: schools_distance, Length: 377185, dtype: object

In [165]:
data['homeFacts'].value_counts().nlargest(15)

homeFacts
{'atAGlanceFacts': [{'factValue': '', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': '', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': '—', 'factLabel': 'lotsize'}, {'factValue': None, 'factLabel': 'Price/sqft'}]}                 7174
{'atAGlanceFacts': [{'factValue': None, 'factLabel': 'Year built'}, {'factValue': None, 'factLabel': 'Remodeled year'}, {'factValue': None, 'factLabel': 'Heating'}, {'factValue': None, 'factLabel': 'Cooling'}, {'factValue': None, 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': None, 'factLabel': 'Price/sqft'}]}      3532
{'atAGlanceFacts': [{'factValue': '', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': '', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': '', 'factLabe

In [73]:
new_home_facts_cols = ['year_built', 'remodeled_year', 'heating', 'cooling', 'parking', 'lotsize', 'price/sqft']

for index, col in enumerate(new_home_facts_cols):
    data[col] = data['homeFacts'].apply(lambda x: eval(x).get('atAGlanceFacts')[index].get('factValue'))

In [152]:
data['lotsize1'] = data['lotsize']

In [153]:
data['lotsize1'] = data['lotsize1'].str.lower()

In [156]:
data['lotsize1'] = data['lotsize1'].apply(lambda x: 'unkown' if  x == '' or x == '—' or x == '-- sqft lot' or x == 'no data' or x == '-1742' or x == '1' or x == '49,299 ,44' else x)

In [155]:
data['lotsize1'] = data['lotsize1'].apply(lambda x: 'unkown' if '49,299' in x else x)


TypeError: argument of type 'NoneType' is not iterable

In [157]:
data['lotsize1'] = data['lotsize1'].fillna('unkown')

In [158]:
data['lotsize1'].isnull().mean()*100

0.0

In [159]:
data['lotsize1'] = data['lotsize1'].str.replace(',', '.')

In [164]:
data['lotsize1'].sort_values(ascending=False).unique().tolist()

['unkown',
 '9999',
 '9998',
 '9997 sqft',
 '9997',
 '9996 sqft',
 '9996',
 '9995 sqft',
 '9995',
 '9994',
 '9993',
 '9990',
 '999.00 acres',
 '999 sqft',
 '999 sq. ft.',
 '999',
 '9989',
 '9988 sqft',
 '9988',
 '9986',
 '9985',
 '9984 sqft',
 '9984',
 '9983',
 '9982',
 '9980',
 '998 sqft lot',
 '998 sqft',
 '9979',
 '9976',
 '99752',
 '9975 sqft',
 '9975',
 '9974',
 '9973',
 '99725',
 '9971',
 '9970 sqft',
 '997 sqft',
 '9969',
 '9967',
 '9966 sqft',
 '9966',
 '9963',
 '996217',
 '9962 sqft',
 '9962',
 '9961',
 '9960',
 '996 sqft lot',
 '996 sqft',
 '996 sq. ft.',
 '9958',
 '9957 sqft',
 '9950',
 '995 sqft',
 '995',
 '9949',
 '9948 sqft',
 '9948',
 '9945',
 '9944',
 '9941',
 '9940',
 '994 sqft',
 '9938',
 '9937',
 '993604',
 '9933',
 '9932',
 '99317',
 '993168',
 '9931 sqft',
 '9931',
 '9930',
 '993 sqft lot',
 '993 sqft',
 '993 sq. ft.',
 '9928',
 '9927',
 '9926',
 '9923',
 '992297',
 '9922',
 '9920',
 '992 sqft lot',
 '992 sqft',
 '992 sq. ft.',
 '992',
 '9918 sqft',
 '9915',
 '9914

In [161]:
data['lotsize1']

0              unkown
1           5828 sqft
2          8.626 sqft
3          8.220 sqft
4         10.019 sqft
             ...     
377180     8.500 sqft
377181         unkown
377182     1.600 sqft
377183         unkown
377184     6.969 sqft
Name: lotsize1, Length: 377185, dtype: object

In [ ]:
nums = re.findall(r'\d+', s)

In [162]:
def get_lotsize1(lotsize):
#    str_lotsize = str(lotsize)
    for size in lotsize:
        if lotsize == 'unkown':
            return 'unkown'
        elif 'acr' in lotsize:
            acres = re.findall(r'\d*\.\d+|\d+', lotsize)
            area = ' '.join(map(str, acres))
            area = float(area)*43560
            return area
        elif 'sq' in lotsize:
            sqft = re.findall(r'\d*\.\d+|\d+', lotsize)
            area = ' '.join(map(str, sqft))
            return area

In [104]:
def get_lotsize(lotsize):
    str_lotsize = str(lotsize)
    for size in str_lotsize:
        if str_lotsize == 'unkown':
            return 'unkown'
        elif 'acr' in str_lotsize:
            acres = str_lotsize.replace(r"[^\d\.]", "", regex=True)
            area = float(acres)*43560
            return area
        elif 'sq' in str_lotsize:
            area = str_lotsize.replace(r"[^\d\.]", "", regex=True)
            return area
        

In [163]:
data['lotsize1'] = data['lotsize1'].apply(get_lotsize1)

ValueError: could not convert string to float: '49.299 .44'

In [132]:
data['lotsize1']

0         unkown
1           5828
2          8.626
3          8.220
4         10.019
           ...  
377180     8.500
377181    unkown
377182     1.600
377183    unkown
377184     6.969
Name: lotsize1, Length: 377185, dtype: object

In [19]:
data['lotsize'].value_counts().nlargest(15)

TypeError: unhashable type: 'list'

In [35]:
def get_property_type(property_type):
    str_property_type = str(property_type)
    for type in str_property_type:
        if property_type is np.nan:
            return np.nan
        elif 'detached' in str_property_type:
            return 'detached'
        elif 'single-family' in str_property_type or 'single family' in str_property_type:
            return 'single-family'
        elif 'multi-family' in str_property_type or 'multi family' in str_property_type:
            return 'multi-family'
        elif 'condo' in str_property_type:
            return 'condo'
        elif 'lot' in str_property_type:
            return 'lot'
        elif 'townhouse' in str_property_type:
            return 'townhouse'
        else:
            return 'other'

In [36]:
data['propertyType'] = data['propertyType'].apply(get_property_type)

In [37]:
data['propertyType'].value_counts()

propertyType
single-family    186803
condo             51477
other             46989
lot               20572
townhouse         18571
multi-family      12218
detached           5822
Name: count, dtype: int64

In [17]:
def get_status(status):
    str_status = str(status)
    for type in str_status:
        if str_status is np.nan:
            return np.nan
        elif 'for sale' in str_status:
            return 'for sale'
        elif 'under contract' in str_status:
            return 'under contract'
        elif 'active' in str_status:
            return 'active'
        elif 'new' in str_status:
            return 'new'
        elif 'pending' in str_status:
            return 'pending'
        elif 'foreclosed' in str_status or 'foreclosure' in str_status:
            return 'foreclosed'
        elif 'auction' in str_status:
            return 'auction'
        elif 'coming soon' in str_status:
            return 'coming soon'
        else:
            return 'other'

In [18]:
data['status'] = data['status'].str.lower()

In [19]:
data['status'] = data['status'].apply(get_status)

In [20]:
data['status'].describe()

count       377185
unique           9
top       for sale
freq        199571
Name: status, dtype: object

In [21]:
data['status'].value_counts().nlargest(20)

status
for sale          199571
active            105837
other              44096
foreclosed         10909
new                 6165
pending             5364
under contract      3661
auction             1472
coming soon          110
Name: count, dtype: int64

In [15]:
data['beds'] = data['beds'].fillna(data['beds'].mode()[0])

In [16]:
data[data.beds.str.contains('acr',regex=True)]

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
279,NaN,NaN,NaN,127 Grapeland Ave,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,San Antonio,"[{'rating': ['3/10', '4/10', '3/10'], 'data': ...",NaN,78214,0.25 acres,TX,NaN,NaN,NaN,1368899,"$46,000"
491,NaN,NaN,NaN,Martin Anderson Beac,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,Orlando,"[{'rating': ['6/10', '4/10', '5/10'], 'data': ...",NaN,32824,0.44 acres,FL,NaN,NaN,NaN,O5731775,"$495,000"
548,NaN,NaN,NaN,7000 E Broadway Ave,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,Tampa,"[{'rating': ['2/10', '3/10', '4/10'], 'data': ...",NaN,33619,1.43 acres,FL,NaN,NaN,NaN,T3198357,"$349,900"
734,NaN,NaN,NaN,950 Granville Dr,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,Houston,"[{'rating': ['2/10', '2/10', '2/10'], 'data': ...",NaN,77091,0.32 acres,TX,NaN,NaN,NaN,58400586,"$75,000"
1205,NaN,NaN,NaN,4902 Timberlan St,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,Tampa,"[{'rating': ['6/10', '6/10', '5/10'], 'data': ...",NaN,33624,9.7 acres,FL,NaN,NaN,NaN,T2288006,"$1,650,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376060,NaN,NaN,NaN,0 Edgerton Ave,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,Orlando,"[{'rating': ['5/10'], 'data': {'Distance': ['5...",NaN,32833,1.03 acres,FL,NaN,NaN,NaN,O5801978,"$99,000"
376596,NaN,NaN,NaN,00 Trumbo Rd,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,San Antonio,"[{'rating': ['NA', '3/10'], 'data': {'Distance...",NaN,78264,23.72 acres,TX,NaN,NaN,NaN,1358450,"$259,000"
376719,NaN,NaN,NaN,606 Casasanta Trl,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,Austin,"[{'rating': ['10/10', '10/10', '8/10'], 'data'...",NaN,78738,2.72 acres,TX,NaN,NaN,NaN,6672739,"$655,000"
376732,NaN,NaN,NaN,925 Tarryall Dr LOT 22,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,Dallas,"[{'rating': ['3/10', '2/10', '2/10'], 'data': ...",NaN,75224,0.28 acres,TX,NaN,NaN,NaN,14211495,"$37,500"


In [23]:
data['beds'] = data['beds'].apply(lambda x: np.nan if 'sqft' in x  or 'acr' in x else x)

TypeError: argument of type 'float' is not iterable

In [26]:
data['beds'] = data['beds'].fillna(data['beds'].mode()[0])

In [21]:
data[data.beds.str.contains('sqft',regex=True)]

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target


In [63]:
# заменяем на np.nan строки с площадью
data['beds'] = data['beds'].fillna('unkown')
data['beds'] = data['beds'].apply(lambda x: np.nan if 'sqft' in x or 'acr' in x or 'unkown' in x else x)
# оставляем только числа
data['beds'] = data['beds'].str.replace(r"[^\d\.]", "", regex=True)
data['beds'] = data['beds'].apply(lambda x: np.nan if  x == '' else x)

In [64]:
data['beds'] = data['beds'].fillna(data['beds'].mode()[0])

In [65]:
data['beds'] = data['beds'].apply(lambda x: str(x).replace('.', '') if  str(x).endswith('.') else x)

In [60]:
def del_symbol(beds):
    for i in beds:
        if beds.endswith('.'):
            beds.replace('.', '')
    return beds

In [61]:
data['beds'] = data['beds'].apply(del_symbol)

In [52]:
def delete_beds(beds):
    for i in beds:
        if 'sqft' in beds or 'acr' in beds:
            return np.nan
        else:
            return beds
            

In [53]:
data['beds'] = data['beds'].apply(delete_beds)

In [28]:
data['beds'] = data['beds'].str.replace(r"[^\d\.]", "", regex=True)

In [30]:
data['beds'].value_counts().nlargest(20)

beds
3      189026
4       63713
2       47715
5       20328
        17484
3.0      8088
6        6272
1        5692
4.0      5231
2.0      2430
7        1758
5.0      1372
0        1230
8        1178
9         478
1.0       429
6.0       311
10        263
0.0       223
12        189
Name: count, dtype: int64

In [37]:
data['beds'] = data['beds'].fillna(data['beds'].mode()[0])

In [66]:
data['beds'].unique().tolist()

['4',
 '3',
 '5',
 '2',
 '8',
 '2.0',
 '1',
 '3.0',
 '6',
 '12',
 '14',
 '7',
 '5.0',
 '4.0',
 '9',
 '0',
 '7.0',
 '16',
 '1.0',
 '6.0',
 '13',
 '10',
 '18',
 '20',
 '0.0',
 '32',
 '11',
 '26',
 '64',
 '144',
 '22',
 '24',
 '15',
 '10.0',
 '8.0',
 '28',
 '36',
 '19',
 '48',
 '25',
 '17',
 '27',
 '40',
 '47',
 '23',
 '44',
 '12.0',
 '30.0',
 '33',
 '9.0',
 '14.0',
 '13.0',
 '31',
 '16.0',
 '34',
 '30',
 '99',
 '78',
 '36073122',
 '11.0',
 '53',
 '27.0',
 '51',
 '21',
 '75',
 '29',
 '15.0',
 '39',
 '42',
 '70824812',
 '52',
 '46',
 '76',
 '35',
 '88']

In [39]:
data['beds'].isnull().mean()*100

0.0

In [43]:
data['beds'] = data['beds'].filter( lambda x: not str(x).endswith('.'), data['beds'] )

TypeError: Keyword arguments `items`, `like`, or `regex` are mutually exclusive

In [44]:
def del_symbol(beds):
    for i in beds:
        if beds.endswith('.'):
            beds.remove('.')
    return beds

In [45]:
data['beds'] = data['beds'].apply(del_symbol)

In [46]:
data['beds'].unique().tolist()

['np.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nannnp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nanpnp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nan.np.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nannnp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nananp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nannnp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nan4np.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nannnp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nanpnp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nan.np.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nannnp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nananp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nannnp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nan',
 'np.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nannnp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nanpnp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nan.np.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nannnp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nananp.nannnp.nanpnp.nan.np.nannnp.nananp.nannnp.nannnp.nannnp.nanp

In [38]:
data['beds'] = data['beds'].apply(lambda x: str(x).endswith('.').str(x).replace('.', ''))

AttributeError: 'bool' object has no attribute 'str'

In [36]:
data['beds'] = data['beds'].apply(lambda x: str(x).replace('', 'np.nan'))
data['beds'] = data['beds'].apply(lambda x: str(x).endswith('.').str(x).replace('.', ''))

AttributeError: 'bool' object has no attribute 'str'

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (15,4))
histplot = sns.histplot(data = data, x = 'target', ax = axes[0]);
histplot.set_title('Распределение целевого признака до удаления выбросов')
boxplot = sns.boxplot( data = data, x = 'target', ax = axes[1])
boxplot.set_title('Целевой признак до удаления выбросов')

In [ ]:
qqplot(data.target, line='s')
plt.title('Квантиль-квантиль график \n для распределения стоимости недвижимости')
plt.show()

In [ ]:
alpha=0.5
_,p = stats.normaltest(data['target'])
print('p-value = %.3f'%(p))
if p <= alpha:
    print('p-значение меньше, чем заданный уровень значимости {:.1f}. Распределение отлично от нормального'.format(alpha))
else:
    print('p-значение больше, чем заданный уровень значимости {:.1f}. Распределение является нормальным'.format(alpha))

In [ ]:
outliers, data = outliers_z_score(data, 'target', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {data.shape[0]}')

In [ ]:
fig, axes = plt.subplots(1,2, figsize = (15,4))
histplot = sns.histplot(data = data, x = 'target', ax = axes[0]);
histplot.set_title('Распределение целевого признака после удаления выбросов')
boxplot = sns.boxplot( data = data, x = 'target', ax = axes[1])
boxplot.set_title('Целевой признак после удаления выбросов')

In [5]:
data = data.dropna(subset=['target'])

In [6]:
data[data.target.str.contains('/mo',regex=True)]

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
547,for rent,NaN,single-family home,4323 N Central Park Ave,3.5 Baths,"{'atAGlanceFacts': [{'factValue': '1913', 'fac...",yes,Chicago,"[{'rating': ['1/10', '4/10', '2/10', 'None/10'...","3,300 sqft",60618,4 Beds,IL,NaN,NaN,NaN,10588057,"$5,500/mo"
609,for rent,NaN,multi-family,220 Boylston St #1412,2 Baths,"{'atAGlanceFacts': [{'factValue': '1985', 'fac...",yes,Boston,"[{'rating': [], 'data': {'Distance': [], 'Grad...","1,673 sqft",2116,2 Beds,MA,NaN,NaN,NaN,72580936,"$10,500/mo"
2075,for rent,NaN,single-family home,2830 NE 56th Ct,4 Baths,"{'atAGlanceFacts': [{'factValue': '1965', 'fac...",NaN,Fort Lauderdale,"[{'rating': ['6/10', '2/10', '4/10'], 'data': ...","2,400 sqft",33308,4 Beds,FL,NaN,NaN,yes,A10521855,"$6,390/mo"
3025,for rent,NaN,multi-family,411 Kline Aly,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,Clarksville,"[{'rating': ['8/10', '9/10', '7/10'], 'data': ...","1,280 sqft",37040,2 Beds,TN,NaN,NaN,NaN,2102821,"$1,200/mo"
3645,for rent,NaN,multi-family,240 E Illinois St #2011,2 Baths,"{'atAGlanceFacts': [{'factValue': '2003', 'fac...",NaN,Chicago,"[{'rating': ['4/10', '7/10'], 'data': {'Distan...","1,473 sqft",60611,2 Beds,IL,NaN,NaN,yes,10590275,"$3,600/mo"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371791,for rent,NaN,multi-family,9436 Turrentine Dr,1.5 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,El Paso,"[{'rating': ['4/10', '8/10', '6/10'], 'data': ...","1,050 sqft",79925,2 Beds,TX,NaN,NaN,NaN,820163,$890/mo
372459,for rent,NaN,townhouse,34 Jonquil Pl,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,The Woodlands,"[{'rating': ['5/10', '8/10', '7/10', '8/10'], ...","2,601 sqft",77375,3 Beds,TX,NaN,NaN,NaN,62158637,"$2,500/mo"
374288,for rent,NaN,single-family home,8864 Devonshire Dr,2 Baths,"{'atAGlanceFacts': [{'factValue': '2016', 'fac...",yes,Fort Worth,"[{'rating': ['6/10', '5/10', '5/10'], 'data': ...","2,000 sqft",76131,4 Beds,TX,NaN,NaN,NaN,NaN,"$2,000/mo"
375550,for rent,NaN,townhouse,2217 W Seybert St,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Philadelphia,"[{'rating': ['1/10', '3/10'], 'data': {'Distan...",720 sqft,19121,2 Beds,PA,NaN,NaN,NaN,PAPH857944,"$1,500/mo"


In [ ]:
# выведем статистику
data['target'].describe()

In [ ]:
# посмотрим на распределение
data['target'].value_counts().nlargest(15)

In [ ]:
data['target'].isnull().mean()*100

In [ ]:
# визуализируем распределение недвижимости по типу отопления
data_heating = data.groupby('heating')['heating'].value_counts()
data_heating = data_heating.sort_values(ascending=False)
# строим древовидную диаграмму с рапределением типов отопления
plt.figure(figsize=(22,6), dpi= 80)
squarify.plot(sizes = data_heating.values, label = data_heating.index, alpha=.6, color = ['red','orange','lightblue','green','pink','yellow', 'blue', 'white'])

plt.title('Статусы продажи')
plt.axis('off')
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(22, 5))

# Строим столбчатую диаграмму для визуализации средней стоимости недвижимости в зависимости от типа отопления
sns.barplot(
            data, 
            x='heating', 
            y='target', 
            hue='heating', 
            legend=False, 
            ax=axes[0]
            )
axes[0].set(xlabel='Тип отопления', ylabel='Средняя стоимость недвижимости')
axes[0].xaxis.set_tick_params(rotation = 90)
axes[0].set_title('Средняя стоимость недвижимости в зависимости от типа отопления')

# Строим коробчатую диаграмму для визуализации стоимости недвижимости в зависимости от типа отопления
sns.boxplot(
            data, 
            x='target', 
            y='heating', 
            orient='h', 
            hue='heating',
            legend=False, 
            medianprops={"color": "red", "linestyle": '--'}
            )
axes[1].set(xlabel='Стоимость недвижимости', ylabel='Тип отопления')
axes[1].set_title(' Стоимость недвижимости в зависимости от типа отопления')

plt.show()

In [ ]:
encoder = ce.OneHotEncoder(cols = ['heating'], use_cat_names=True)
heating_bin = encoder.fit_transform(data['heating'])
data = pd.concat([data, heating_bin], axis=1)

In [ ]:
'--'

In [34]:
data['street'].value_counts().nlargest(15)

street
Address Not Disclosed         672
Undisclosed Address           517
(undisclosed Address)         391
Address Not Available         175
Unknown Address                72
2103 E State Hwy 21            57
11305 Gulf Fwy                 54
17030 Youngblood Rd.           38
NE 58th Cir                    34
9470 Lancaster Rd. SW          32
1 Palmer Dr                    27
8426 Terrace Valley Circle     25
9845 Basil Western Rd NW       25
6320 SW 89th Court Road        24
8447 SW 99th Street Rd         22
Name: count, dtype: int64

In [69]:
data['street'] = data['street'].astype(str)
data['street'] = data['street'].str.lower()

In [70]:
# удалим ненужные символы и переведем в тип данных float
data['street'] = data['street'].apply(lambda x: str(x).replace('(', '').replace(')', ''))

In [56]:
data['street'].unique().tolist()

['240 heather ln',
 '12911 e heroy ave',
 '2005 westridge rd',
 '4311 livingston ave',
 '1524 kiscoe st',
 '1624 s newkirk st',
 '552 casanova ct ',
 '6094 mingle dr ',
 '11182 owl ave',
 '8612 cedar plains ln',
 '14205 roosevelt ave #512',
 '4222 false cypress ln ',
 '16373 nw 16th st',
 '1538 umpqua dr',
 '15351 falmouth ave',
 '930 52nd st ',
 '2402 nw 22nd rd ',
 '24 w 126th st ',
 '22702 alderdale ln',
 '1633 ne 15th st',
 '5930 e avenue t2',
 '61425 gosney rd',
 '120 jefferson ave #12019 ',
 '4234 orchard pass dr',
 '703 w gemini ln',
 '1755 18th st nw',
 '5755 sw 123 ave ',
 '2104 lou ellen ln #b',
 '9070 southern orchard rd s',
 '430 sterling pl #1a',
 '2620 alsace ave',
 '6712 n glen ave',
 '2043 se new york st',
 '3750 ne 170th st apt 407',
 '7314 melvin cir n',
 '423 n church st #13-a',
 '2627 n arthur ave',
 '3360 sw 29th st',
 '5531 eureka springs rd',
 '316 n henderson ave #4',
 '1104 rotonda cir ',
 '9418 camino santa fe #19',
 'mls #: stri1057289',
 '2713 oakmont ct',
 

In [71]:
data['street'].value_counts().nlargest(15)

street
undisclosed address           908
address not disclosed         672
address not available         175
unknown address                72
2103 e state hwy 21            57
11305 gulf fwy                 54
17030 youngblood rd.           38
ne 58th cir                    34
9470 lancaster rd. sw          32
1 palmer dr                    27
8426 terrace valley circle     25
9845 basil western rd nw       25
6320 sw 89th court road        24
8447 sw 99th street rd         22
5221 s. zapata hwy             20
Name: count, dtype: int64

In [72]:
def get_type_of_street(street):
    for type in street:
        if street is np.nan:
            return np.nan
        elif 'address not disclosed' in street or 'undisclosed address' in street:
            return 'undisclosed'
        elif 'address not available' in street:
            return 'not available'
        elif 'unknown address' in street:
            return 'unkown'
        elif 'parkway' in street or 'prwy' in street:
            return 'prwy'
        elif 'boulevard' in street or 'blw' in street or'blwd' in street:
            return 'blw'
        elif 'railway' in street or 'rw' in street or'rwy' in street:
            return 'rwy'
        elif 'quay' in street or 'qy' in street:
            return 'qy'
        elif 'lane' in street or 'ln' in street:
            return 'ln'
        elif 'square' in street or 'sq' in street:
            return 'sq'
        elif 'place' in street or 'pl' in street:
            return 'pl'
        elif 'drive' in street or 'dr' in street:
            return 'dr'
        elif 'avenue' in street or 'ave' in street:
            return 'ave'
        elif 'terrace' in street or 'tce' in street:
            return 'tce'
        elif 'street' in street or 'st' in street:
            return 'st'
        elif 'hill' in street or 'hl' in street:
            return 'hl'
        elif 'road' in street or 'rd' in street:
            return 'rd'
        elif 'highway' in street or 'hwy' in street:
            return 'hwy'
        else:
            return 'unkown'

In [73]:
data['street'] = data['street'].apply(get_type_of_street)

In [74]:
data['street'].value_counts().nlargest(20)

street
st               91464
unkown           69446
dr               64856
ave              63495
rd               31623
ln               24672
pl               23312
hl                2061
rwy               1971
undisclosed       1580
hwy               1396
sq                 572
tce                234
not available      175
blw                162
prwy               119
qy                  47
Name: count, dtype: int64

In [ ]:
da = df[df.col1 > 8]

In [9]:
data = data.drop(data[data['status'] != 'for sale'].index)

In [10]:
data[data.target.str.contains('/mo',regex=True)]

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target


In [8]:
data = data(data['status'] != 'for sale')

TypeError: 'DataFrame' object is not callable